# ETL-Prozess: PV-Ertrag
---
Diese Komponente ist für das Extrahieren, Transformieren und Laden von Daten aus der Quellen [Deutscher Wetterdienst](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/solar/BESCHREIBUNG_obsgermany_climate_hourly_solar_de.pdf) und [Photovoltaic Geographical Information System](https://joint-research-centre.ec.europa.eu/photovoltaic-geographical-information-system-pvgis_en) zuständig. Die Komponente verarbeitet Quelldateien mit Globalstrahlungsdaten und simulierten PV-Ertragswerten.

# Funktionsweise des Prozesses
---
In diesem Prozess werden die folgenden Funktionen verwendet:

1. `extraction_dataset_dwd()`: Diese Funktion extrahiert die Daten aus einer [Quelldatei](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/solar/stundenwerte_ST_01975_row.zip), die den stündlichen Globalstrahlungsverlauf in Hamburg von 2005 bis heute enthält.


2. `processing_dataset_dwd()`: Die Funktion verarbeitet die Globalstrahlungsdaten und gibt sie in 15-Minuten-Intervallen weiter.


3. `extraction_dataset_pvgis()`: Diese Funktion extrahiert Daten aus einer Quelldatei, die im [PVGIS-Simulator](https://re.jrc.ec.europa.eu/pvg_tools/de/#SA) definierte Parameter enthält. Diese Werte werden verwendet, um die Stromerzeugung von PV-Anlagen in 15-Minuten-Intervallen zu berechnen.


4. `processing_dataset_pvgis()`: Diese Funktion berechnet die Stromerzeugung der PV-Anlagen in 15-Minuten-Intervallen. Die Berechnungen beziehen sich auf PV-Anlagen mit 10-kWp, 20-kWp und 30-kWp Gesamtkapazität.


5. `run_ETL_PV_Ertrag()`: Funktion zur Ausführung des Prozesses.

# Quellcode der Funktionen
---
Hier befindet sich der Code für die Funktionen, die im Abschnitt "Funktionsweise des Prozesses" beschrieben sind.

In [1]:
import pandas as pd
import os
import openpyxl
from datetime import datetime

def extraction_dataset_dwd():
    try:
        # Erhaltung des aktuellen Arbeitsverzeichnisses
        cwd = os.getcwd()
        input_folder = "Datenquelle/DWD-Daten/Stundenwerte_Globalstrahlung"
        # Festlegung des Verzeichnisses, in dem sich die Datei befindet
        data_directory = os.path.join(cwd, input_folder)

        # Angeben des Dateinamens
        input_file = "produkt_st_stunde_20050101_20240229_01975.txt"
        # Kombinieren von Verzeichnis und Dateiname
        input_path = os.path.join(data_directory, input_file)

        # Überprüfen, ob die Datei vorhanden ist
        if not os.path.exists(input_path):
            raise FileNotFoundError(f"Fehler: Datei '{input_file}' nicht im Verzeichnis '{data_directory}' gefunden.")

        # Lesen der Daten aus der Textdatei und Interpretation von 'MESS_DATUM' als Datum
        data = pd.read_csv(input_path, sep=";", usecols=["MESS_DATUM", "FG_LBERG"], parse_dates=["MESS_DATUM"])

        # Filtern der Daten nach dem Zeitbereich im String-Format
        start_date_str = "2023010101:23"
        end_date_str = "2024010100:23"
        filtered_data = data[(data["MESS_DATUM"] >= start_date_str) & (data["MESS_DATUM"] <= end_date_str)]

        # Öffnen der Excel-Datei und Schreiben der Daten
        output_folder = "Output-Data"
        output_file = "ETL-Prozess_Ergebnisse.xlsx"
        output_path = os.path.join(output_folder, output_file)

        with pd.ExcelWriter(output_path, engine='openpyxl', mode='a') as writer:
            filtered_data.to_excel(writer, index=False, sheet_name="DWD-Data")

        print("- ETL-Prozess_PV-Ertrag: Aufgabe 1/4 abgeschlossen")

    except FileNotFoundError as e:
        print(f"Fehler beim Lesen der Datei: {e}")

    except Exception as e:
        print(f"Fehler beim Verarbeiten der Daten: {e}")
        

def extraction_dataset_pvgis():
    try:
        # Angeben von Dateinamen und Blattnamen
        input_folder = "Datenquelle/PVGIS-Daten/Solardatenparameter_2023"
        input_file = "PVGIS-Daten_AVG-PV-Ertrag 2023.xlsx"
        output_folder = "Output-Data"
        output_file = "ETL-Prozess_Ergebnisse.xlsx"
        input_sheet_name = "PVGIS-Daten_AVG-PV-Ertrag 2023"
        output_sheet_name = "PVGIS-Data"

        # Kombinieren der Verzeichnisse
        input_path = os.path.join(input_folder, input_file)
        output_path = os.path.join(output_folder, output_file)

        # Überprüfen, ob die Eingabedatei vorhanden ist
        if not os.path.exists(input_path):
            raise FileNotFoundError(f"Fehler: Datei '{input_file}' nicht im Verzeichnis '{input_folder}' gefunden.")

        # Lesen der Daten aus der Quelldatei
        data = pd.read_excel(input_path, sheet_name=input_sheet_name)

        # Öffnen der Excel-Datei und Schreiben der Daten
        with pd.ExcelWriter(output_path, engine='openpyxl', mode='a') as writer:
            data.to_excel(writer, index=False, sheet_name=output_sheet_name)

        print("- ETL-Prozess_PV-Ertrag: Aufgabe 3/4 abgeschlossen")

    except FileNotFoundError as e:
        print(f"Fehler beim Lesen der Datei: {e}")

    except Exception as e:
        print(f"Fehler beim Verarbeiten der Daten: {e}")
        
def processing_dataset_dwd():
    try:
        # Festlegung der Pfade für die Eingabe- und Ausgabedateien
        input_folder = "Output-Data"
        input_file = "ETL-Prozess_Ergebnisse.xlsx"
        input_path = os.path.join(input_folder, input_file)

        # Öffnen der Excel-Datei im try-Block
        workbook = openpyxl.load_workbook(input_path)

        # Auswahl des "DWD-Daten"-Blatts und Extrahieren der Werte aus der Spalte "FG_LBERG"
        dwd_sheet = workbook['DWD-Data']
        fg_lberg_values = [cell.value for cell in dwd_sheet['B'][1:]]

        # Überprüfen, ob der Wert kleiner als 0 ist, falls ja, dann auf den Wert der vorherigen Zelle setzen
        for i in range(len(fg_lberg_values)):
            if fg_lberg_values[i] is not None and fg_lberg_values[i] < 0:
                fg_lberg_values[i] = fg_lberg_values[i-1] if i > 0 else None

        # Auswahl des "Simulationsparameter"-Blatts und Erstellen einer neuen Spalte "Globalstrahlung"
        etl_sheet = workbook['Simulationsparameter']
        etl_sheet['E1'] = 'Globalstrahlung'

        # Schreiben der Werte der Spalte "Globalstrahlung" basierend auf den bereinigten Werten von "FG_LBERG"
        global_radiation_values = []
        for fg_lberg_value in fg_lberg_values:
            if fg_lberg_value is not None:
                # Teilen des Werts von "FG_LBERG" durch 4 und Hinzufügen der Ergebnisse zur Liste
                global_radiation_values.extend([fg_lberg_value / 4] * 4)
            else:
                # Falls der Wert von "FG_LBERG" None ist, Hinzufügen von 4 leeren Zellen
                global_radiation_values.extend([None] * 4)

        # Schreiben der Werte in die Spalte "Globalstrahlung"
        for row, value in enumerate(global_radiation_values, start=2):
            etl_sheet.cell(row=row, column=5, value=value)

        # Speichern der Änderungen in der Excel-Datei
        workbook.save(input_path)

        print("- ETL-Prozess_PV-Ertrag: Aufgabe 2/4 abgeschlossen")

    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")
    
def processing_dataset_pvgis():
    try:
        # Festlegung der Pfade für die Eingabe- und Ausgabedateien
        output_folder = "Output-Data"
        input_folder = output_folder
        input_file = "ETL-Prozess_Ergebnisse.xlsx"
        input_path = os.path.join(input_folder, input_file)

        # Öffnen der Excel-Datei im try-Block
        workbook = openpyxl.load_workbook(input_path)

        # Auswahl des "Simulationsparameter"-Blatts
        sheet = workbook['Simulationsparameter']

        # Hinzufügen von Überschriften für neue Spalten
        sheet['F1'] = 'PV-10kWp'
        sheet['G1'] = 'PV-20kWp'
        sheet['H1'] = 'PV-30kWp'
        
        # Hinzufügen von Spalten für Stromspeicher, Wallbox und Wärmepumpe
        sheet['I1'] = 'Stromspeicher'
        sheet['J1'] = 'Wallbox'
        sheet['K1'] = 'Wärmepumpe'
        
        # Erstellen eines leeren Dictionaries zur Speicherung der monatlichen Gesamtsummen
        monthly_totals = {}

        # Iteration über die Zeilen der Excel-Datei, um die monatliche globale Strahlung aufsummiert zu berechnen
        for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=5):
            datum = row[0].value
            globalstrahlung = row[4].value
            
            if isinstance(datum, str):
                datum = datetime.strptime(datum, '%Y-%m-%d')

            # Überprüfen, ob das Datum gültig ist
            if datum is not None:
                # Extrahieren des Monats aus dem Datum
                month = datum.strftime('%B')

                # Hinzufügen des Monats zum monatlichen Gesamtsummen-Dictionary, falls noch nicht vorhanden
                if month not in monthly_totals:
                    monthly_totals[month] = {'sum_fg_lberg': 0}

                # Überprüfen, ob die globale Strahlung gültig ist
                if globalstrahlung is not None:
                    # Kumulative Summe der globalen Strahlung für jeden Monat berechnen
                    monthly_totals[month]['sum_fg_lberg'] += globalstrahlung

        # Laden der Werte von "gesamtmonat_kWp" aus der Tabelle "PVGIS-Daten_AVG-PV-Ertrag 2023"
        total_month_kwp = {}
        sheet_performance = workbook['PVGIS-Data']
        for row in range(2, sheet_performance.max_row + 1):
            month_performance = sheet_performance[f'A{row}'].value
            total_10kwp = sheet_performance[f'C{row}'].value
            total_20kwp = sheet_performance[f'E{row}'].value
            total_30kwp = sheet_performance[f'G{row}'].value
            total_month_kwp[month_performance] = {'10kWp': total_10kwp, '20kWp': total_20kwp, '30kWp': total_30kwp}

        # Iteration über die Monate und Berechnung der Werte für die neuen Spalten "10 kWP", "20 kWP" und "30 kWP"
        for month, data in monthly_totals.items():
            sum_globalstrahlung = data['sum_fg_lberg']
            # Überprüfen, ob der Monat in "total_month_kwp" vorhanden ist
            if month in total_month_kwp:
                gesamtmonat_kwp = total_month_kwp[month]
                # Berechnung und Einfügen der Werte für "10 kWP", "20 kWP" und "30 kWP" in die neuen Spalten
                for row in range(2, sheet.max_row + 1):
                    datum = sheet[f'A{row}'].value
                    if isinstance(datum, str):
                        datum = datetime.strptime(datum, '%Y-%m-%d')
                    globalstrahlung = sheet[f'E{row}'].value
                    if datum is not None and datum.strftime('%B') == month:
                        ten_kwp = (globalstrahlung * gesamtmonat_kwp['10kWp']) / sum_globalstrahlung
                        twenty_kwp = (globalstrahlung * gesamtmonat_kwp['20kWp']) / sum_globalstrahlung
                        thirty_kwp = (globalstrahlung * gesamtmonat_kwp['30kWp']) / sum_globalstrahlung
                        sheet[f'F{row}'] = ten_kwp
                        sheet[f'G{row}'] = twenty_kwp
                        sheet[f'H{row}'] = thirty_kwp
            else:
                print(f"Warnung: Keine Daten für den Monat {month} in 'PVGIS-Data' gefunden.")    

        # Speichern der Änderungen in der Eingabedatei
        workbook.save(input_path)

        print("- ETL-Prozess_PV-Ertrag: Aufgabe 4/4 abgeschlossen")

    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

In [2]:
def run_ETL_PV_Ertrag():
    extraction_dataset_dwd()
    processing_dataset_dwd()
    extraction_dataset_pvgis()
    processing_dataset_pvgis()